In [1]:

!pip install git+https://github.com/openai/whisper.git

!pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-rolzlcx9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-rolzlcx9
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=f19d4ce1c4174dbf5c977ee5793c76c555539d0580647990986910084b875bbb
  Stored in directory: /tmp/pip-ephem-wheel-cache-2cqz8b1l/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 89.0 MB/s eta 0:00:00


In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
from jiwer import wer, cer

from scipy.io import wavfile
from tqdm.notebook import tqdm
from IPython.display import display, Audio, HTML

pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000

DEVICE = "cuda"
print(f"Using device: {DEVICE} (Google Colab T4 GPU)")

Using device: cuda (Google Colab T4 GPU)


In [ ]:
def download(url: str, target_path: str):
    """Download a file with progress bar"""
    with urllib.request.urlopen(url) as source, open(target_path, "wb") as output:
        with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
            while True:
                buffer = source.read(8192)
                if not buffer:
                    break
                output.write(buffer)
                loop.update(len(buffer))


class Fleurs(torch.utils.data.Dataset):

    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            print(f"Downloading FLEURS dataset for {lang}...")
            download(url, tar_path)
        else:
            print(f"Using cached dataset: {tar_path}")

        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]
        return (audio, text)

In [ ]:
# Configuration for Bahasa Indonesia
LANG_CODE = "id_id"  
LANGUAGE_NAME = "Indonesian"

# Subsample rate: 5 means we take every 5th sample (20% of data) for faster demo
SUBSAMPLE_RATE = 5

print(f"Loading FLEURS dataset for {LANGUAGE_NAME} ({LANG_CODE})...")
dataset = Fleurs(LANG_CODE, subsample_rate=SUBSAMPLE_RATE)
print(f"Dataset loaded: {len(dataset)} samples")

Loading FLEURS dataset for Indonesian (id_id)...


  0%|                                              | 0.00/2.21G [00:00<?, ?iB/s]

Dataset loaded: 138 samples


In [ ]:
# Model selection
MODEL_NAME = "base"
print(f"Loading Whisper {MODEL_NAME} model...")
model = whisper.load_model(MODEL_NAME)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Loading Whisper base model...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 237MiB/s]


Model is multilingual and has 71,825,920 parameters.


In [ ]:
# Transcription options
options = dict(language=LANGUAGE_NAME, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)

# Store results
references = []
transcriptions = []

print(f"Transcribing {len(dataset)} audio samples...")
print(f"Using model: {MODEL_NAME} on device: {DEVICE}")
print("-" * 50)

for audio, text in tqdm(dataset):
    transcription = model.transcribe(audio, **transcribe_options)["text"]

    transcriptions.append(transcription)
    references.append(text)

print(f"\n Completed transcription of {len(transcriptions)} samples.")

Transcribing 138 audio samples...
Using model: base on device: cuda
--------------------------------------------------


  0%|          | 0/138 [00:00<?, ?it/s]


 Completed transcription of 138 samples.


In [ ]:
# Display audio examples
NUM_EXAMPLES = 5  

print("="*70)
print(" AUDIO EXAMPLES WITH TRANSCRIPTION RESULTS")
print("="*70)

for i in range(min(NUM_EXAMPLES, len(dataset))):
    audio, text = dataset[i]

    print(f"\n--- Example {i+1} ---")

    # Play audio and show comparison
    display(Audio(audio.numpy(), rate=16000))
    print(f"\n Ground Truth (Reference):")
    print(f"   {references[i]}")
    print(f"\n Whisper Transcription:")
    print(f"   {transcriptions[i]}")

    # Calculate per-sample metrics
    sample_wer = wer(references[i], transcriptions[i]) * 100
    sample_cer = cer(references[i], transcriptions[i]) * 100
    print(f"\n Sample Metrics: WER = {sample_wer:.2f}%, CER = {sample_cer:.2f}%")
    print("-" * 50)

 AUDIO EXAMPLES WITH TRANSCRIPTION RESULTS

--- Example 1 ---



 Ground Truth (Reference):
   strategi tersebut terbukti efektif memutus pasokan penting militer dan sipil meskipun blokade ini melanggar hukum internasional yang diterima secara umum yang dikodifikasi oleh beberapa perjanjian internasional selama dua abad terakhir

 Whisper Transcription:
    Stratak-kiternya sebut terpukta efektif memutus pasokan penting militer dan cipil. Meskipun plokak ada ini melanggar hukum internasional yang gitari masyarakumum yang dikodifikasi oleh beberapa perjanjian internasional selamat 2 apaterahir.

 Sample Metrics: WER = 46.67%, CER = 15.32%
--------------------------------------------------

--- Example 2 ---



 Ground Truth (Reference):
   tumbuhan membuat makanan mereka dari matahari melalui fotosintesis mereka juga memberikan tempat berteduh

 Whisper Transcription:
    Tumbuhan membuat makanan mereka dari matahari melalui foto sintesis, mereka juga memberikan tempat berteduh.

 Sample Metrics: WER = 30.77%, CER = 3.81%
--------------------------------------------------

--- Example 3 ---



 Ground Truth (Reference):
   namun ada banyak hal tentang burung yang masih terlihat seperti dinosaurus

 Whisper Transcription:
    Namun, ada banyak hal tentang burung yang masih terlihat seperti dinosaurus.

 Sample Metrics: WER = 18.18%, CER = 4.05%
--------------------------------------------------

--- Example 4 ---



 Ground Truth (Reference):
   pada dasarnya logam campuran merupakan campuran dari dua logam atau lebih ingat ada banyak elemen pada tabel periodik

 Whisper Transcription:
    Pada dasarnya lukam tamburan merupakan tamburan dari 2 lukam atau lebih, ingat ada banyak element pada tapel periotic.

 Sample Metrics: WER = 55.56%, CER = 15.38%
--------------------------------------------------

--- Example 5 ---



 Ground Truth (Reference):
   perdebatan dipicu kontroversi terkait pengeluaran dana bantuan dan rekonstruksi pasca-badai katrina yang dilabeli oleh kelompok konservatif fiskal sebagai kesepakatan new orleans bush

 Whisper Transcription:
    Perdebatan dipicuk kontroversi terkait mengeluaran dan abantuan dan rekonstruksi pas kaba daikat rina yang dilabali oleh kelompok konservatif viskal sebagai kesepakatan New Orleans Busch.

 Sample Metrics: WER = 63.64%, CER = 9.84%
--------------------------------------------------


In [13]:
# Create results dataframe
results_df = pd.DataFrame({
    "Reference": references,
    "Transcription": transcriptions
})

# Calculate overall WER and CER
overall_wer = wer(references, transcriptions) * 100
overall_cer = cer(references, transcriptions) * 100

print(" METRIC EVALUATION RESULTS")
print("=" * 50)
print(f"\n Word Error Rate (WER):      {overall_wer:.2f}%")
print(f" Character Error Rate (CER): {overall_cer:.2f}%")
print(f"\n Total samples evaluated: {len(references)}")

 METRIC EVALUATION RESULTS

 Word Error Rate (WER):      49.47%
 Character Error Rate (CER): 15.52%

 Total samples evaluated: 138


In [14]:
# Display results table
print("\n Sample-by-sample Results (first 10 samples):")
print("=" * 50)
display(results_df.head(10))


 Sample-by-sample Results (first 10 samples):


,Reference,Transcription
0,strategi tersebut terbukti efektif memutus pasokan penting militer dan sipil meskipun blokade ini melanggar hukum internasional yang diterima secara umum yang dikodifikasi oleh beberapa perjanjian internasional selama dua abad terakhir,Stratak-kiternya sebut terpukta efektif memutus pasokan penting militer dan cipil. Meskipun plokak ada ini melanggar hukum internasional yang gitari masyarakumum yang dikodifikasi oleh beberapa perjanjian internasional selamat 2 apaterahir.
1,tumbuhan membuat makanan mereka dari matahari melalui fotosintesis mereka juga memberikan tempat berteduh,"Tumbuhan membuat makanan mereka dari matahari melalui foto sintesis, mereka juga memberikan tempat berteduh."
2,namun ada banyak hal tentang burung yang masih terlihat seperti dinosaurus,"Namun, ada banyak hal tentang burung yang masih terlihat seperti dinosaurus."
3,pada dasarnya logam campuran merupakan campuran dari dua logam atau lebih ingat ada banyak elemen pada tabel periodik,"Pada dasarnya lukam tamburan merupakan tamburan dari 2 lukam atau lebih, ingat ada banyak element pada tapel periotic."
4,perdebatan dipicu kontroversi terkait pengeluaran dana bantuan dan rekonstruksi pasca-badai katrina yang dilabeli oleh kelompok konservatif fiskal sebagai kesepakatan new orleans bush,Perdebatan dipicuk kontroversi terkait mengeluaran dan abantuan dan rekonstruksi pas kaba daikat rina yang dilabali oleh kelompok konservatif viskal sebagai kesepakatan New Orleans Busch.
5,rumah sakit telah mengikuti protokol untuk pengendalian infeksi termasuk memisahkan pasien dari orang lain untuk mencegah kemungkinan menulari mereka,"Rumah sakit lalu mengikuti protokol untuk pengen dalam infeksi, termasuk memisahkan pasian dari orang lain untuk menjegal kemungkinan menulari mereka."
6,pengumuman itu dibuat setelah trump melakukan percakapan telepon dengan presiden turki recep tayyip erdoäÿan,Pengemuman itu dibuat setelah teram melakukan percakap antelipon dengan President Turki Reseptive Erduh Khan.
7,setelah bendungan dibangun pada 1963 banjir musiman yang dapat menyebarkan sedimen ke sepanjang sungai berhasil dihentikan,"Setelah bendungan di bangun pada 1963, penjir musimannya yang mendapat menyebarkan sehidimain kesepanjang sungai berhasil dihentikan."
8,area yang luas di sebelah utara sangat jarang ditempati dan beberapa bahkan adalah hutan belantara yang hampir tak dapat ditempati,Arya yang luas di sebelah utara sangat carang di tempati dan beberapa bahkan adalah udan pelantara yang ambil tak dapat di tempati.
9,peradaban adalah sebuah kebudayaan tunggal yang dijalankan bersama oleh sekelompok orang yang hidup dan bekerja bersama sebagai masyarakat,Peradaban adalah sebuah kebudayaan tunggal yang dijelakan bersama oleh sklapok orang yang hidup dan bekerja bersama sebagai masyarakat


In [ ]:
def interpret_wer(wer_score):
    if wer_score < 10:
        return "Excellent", "The model performs exceptionally well on Bahasa Indonesia."
    elif wer_score < 20:
        return "Good", "The model performs well with minor errors."
    elif wer_score < 30:
        return "Fair", "The model captures most content but has noticeable errors."
    elif wer_score < 50:
        return "Moderate", "The model struggles with some aspects of the language."
    else:
        return "Poor", "The model has significant difficulty with this language."

rating, interpretation = interpret_wer(overall_wer)

print("\n" + "="*70)
print(" RESULT SUMMARY")
print("="*70)
print(f"""
┌────────────────────────────────────────────────────────────────┐
│                     WHISPER ASR DEMO RESULTS                   │
├────────────────────────────────────────────────────────────────┤
│  Language:           {LANGUAGE_NAME} (Bahasa Indonesia)             │
│  Model:              Whisper {MODEL_NAME:<10}                        │
│  Dataset:            FLEURS {LANG_CODE}                              │
│  Samples Evaluated:  {len(references):<10}                                │
├────────────────────────────────────────────────────────────────┤
│                         METRICS                                │
├────────────────────────────────────────────────────────────────┤
│  Word Error Rate (WER):       {overall_wer:>6.2f}%                          │
│  Character Error Rate (CER):  {overall_cer:>6.2f}%                          │
├────────────────────────────────────────────────────────────────┤
│                      INTERPRETATION                            │
├────────────────────────────────────────────────────────────────┤
│  Performance Rating: {rating:<12}                              │
│  {interpretation:<62}│
└────────────────────────────────────────────────────────────────┘
""")

print("\n Demo completed successfully!")


 RESULT SUMMARY

┌────────────────────────────────────────────────────────────────┐
│                     WHISPER ASR DEMO RESULTS                   │
├────────────────────────────────────────────────────────────────┤
│  Language:           Indonesian (Bahasa Indonesia)             │
│  Model:              Whisper base                              │
│  Dataset:            FLEURS id_id                              │
│  Samples Evaluated:  138                                       │
├────────────────────────────────────────────────────────────────┤
│                         METRICS                                │
├────────────────────────────────────────────────────────────────┤
│  Word Error Rate (WER):        49.47%                          │
│  Character Error Rate (CER):   15.52%                          │
├────────────────────────────────────────────────────────────────┤
│                      INTERPRETATION                            │
├───────────────────────────────────────────